# Support vector machines

In [38]:
# import libraries

import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns

from matplotlib.backends.backend_pdf import PdfPages
from sklearn.decomposition import PCA

from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.feature_selection import GenericUnivariateSelect, mutual_info_classif, mutual_info_regression, f_regression, f_classif

from sklearn import preprocessing
from sklearn.svm import LinearSVR
from sklearn.svm import LinearSVC
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn import linear_model

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import r2_score
from sklearn.metrics.pairwise import pairwise_kernels

from sklearn import model_selection

from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

## Data pre-processing

In [2]:
# load training data

# load data from csv file
df_train_features = pd.read_csv ('train_features.csv')
df_train_labels = pd.read_csv('train_labels.csv')

# Load test data
df_test_features = pd.read_csv('test_features.csv')

### Sorting labels

In [3]:
df_train_labels = df_train_labels.sort_values('pid')
df_train_features = df_train_features.sort_values(['pid', 'Time'])

# Droping time
df_train_features = df_train_features.drop('Time', axis = 1)
df_test_features = df_test_features.drop('Time', axis = 1)

 ### Histogram of the output labels 

We should check for class imbalance.

In [ ]:
df_train_labels.hist()

# with PdfPages("./Results/Labels_histogram.pdf") as export_pdf:
#     for i in list(df_train_labels)[1:]:
#         df_train_labels.hist(column = i, bins = 100)
#         export_pdf.savefig()

One can see the class imbalance problem here. Other observations:
  * Heartrate, RRate, ABPm,  distribution is similar to a normal distribution
  * SpO2 is like a censored normal distribution. 
  * For all of the other features, class imbalance is an obvious problem.

A basic strategy that could be used here: Upsample both classes! Do the upsampling efficiently, not just replicating the datapoints

### Boxplot over features

In [ ]:
# data inspection: 
#############################################
# range of the provided data?
print(df_train_features.agg([min, max]))

# Boxplotting the data
# fig2, ax2 = plt.subplots()
# ax2.set_title('BUN')
# ax2.boxplot(df_train_features.iloc[:,5], notch=True)

plt.figure(figsize=(16, 16))
ax = sns.boxplot(data = df_train_features.iloc[:,1:])
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=90,
    horizontalalignment='right'
);

# with PdfPages("./Results/Train_columns_boxplot.pdf") as export_pdf:
#     for i in list(df_train_labels)[1:]:
#         df_train_labels.hist(column = i, bins = 100)
#         export_pdf.savefig()

In [ ]:
# calculate the correlation matrix
corr = df_train_features.corr()

# plot the heatmap
plt.figure(figsize=(16, 16))
ax = sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns, 
        vmin=-1, vmax=1, center=0, 
           cmap=sns.diverging_palette(20, 220, n=200))
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

### Visualizing pattern of missing values

In [ ]:
# how much missing data? 
print("Percentage of missing values:")
print(df_train_features.isnull().sum(axis=0) / len(df_train_features))

msno.matrix(df_train_features)

# Plotting the correlation between the missing values
msno.heatmap(df_train_features)

### Train data pre-processing

In [ ]:
# Which columns have less than a percent NA
NA_percent = 0.7
NA_percent_severe = 0.91

sel_features = df_train_features.isnull().sum(axis=0) < (NA_percent * df_train_features.shape[0])
inds = np.where(sel_features == True)

sel_features_2 = (df_train_features.isnull().sum(axis=0) < (NA_percent_severe * df_train_features.shape[0])) & (df_train_features.isnull().sum(axis=0) > (NA_percent * df_train_features.shape[0]))        
inds_2 = np.where(sel_features_2 == True)

sel_features_3 = df_train_features.isnull().sum(axis=0) > (NA_percent_severe * df_train_features.shape[0])
inds_3 = np.where(sel_features_3 == True)

vars_class1 = df_train_features.columns[inds[0]]
vars_class2 = df_train_features.columns[inds_2[0]]
vars_class3 = df_train_features.columns[inds_3[0]]
vars_classes = [vars_class1, vars_class2, vars_class3]

print("--------Our Columns are are: ",df_train_features.columns.drop("pid"))
print("--------Labels are: ", df_train_labels.columns)
print("********Var classes are: " ,vars_classes)

In [ ]:
def aggregation_technique(dat, variables, pids, vars_classes):
    
    vars_class1 = vars_classes[0]
    vars_class2 = vars_classes[1]
    vars_class3 = vars_classes[2]
    
    aggregated_dataframe = []
    for pid in pids:
        # dataframe for this pid
        df = dat.loc[dat['pid'] == pid]
        calculated_features = []
        for var in variables:
            # how many NaN are in there for this variable
            data = df[var].tolist()
#             num_nan = np.count_nonzero(np.isnan(data))
            if var == "Age":
                calculated_features.append(data[0])
                
            elif var in vars_class1:
                calculated_features = calculated_features + data
#                 calculated_features.append(np.nanmean(data))
#                 calculated_features.append(np.nanstd(data))
                
            elif var in vars_class2:
                calculated_features.append(np.nanmean(data))
                calculated_features.append(np.nanstd(data))
                calculated_features.append(np.nanmin(data))
                calculated_features.append(np.nanmax(data))
            else:
                calculated_features.append(np.nanmean(data))
                calculated_features.append(np.nanstd(data))
                
        aggregated_dataframe.append(calculated_features)
        
    aggregated_dataframe = pd.DataFrame(aggregated_dataframe)
    return(aggregated_dataframe)
 
train_pids = df_train_features['pid'].unique()
df_train_agg_features = aggregation_technique(df_train_features, df_train_features.columns.drop("pid"), train_pids, vars_classes)

In [ ]:
print(df_train_agg_features.shape)
print(df_train_agg_features.columns)
print(df_train_agg_features)
pd.DataFrame(data_train_scaled)

In [ ]:
# print(train_pids[1:50])
# print(df_train_labels['pid'].tolist()[1:50])
print(np.nanmean(df_train_features['Age'].tolist()))
print(np.nanvar(df_train_features['Age'].tolist()))

In [ ]:
# # Run the imputer with a simple Random Forest estimator
# imp = IterativeImputer(RandomForestRegressor(n_estimators=5), max_iter=10, random_state=1)

# #perform filling
# df_train_agg_imputed_features = pd.DataFrame(imp.fit_transform(df_train_agg_features), columns=df_train_agg_features.columns)

In [ ]:
# impute missing data points
#imp = SimpleImputer(strategy="mean")
imputer = KNNImputer(n_neighbors = 10)

df_train_agg_imputed_features = imputer.fit_transform(df_train_agg_features)

In [ ]:
# scale the data
min_max_scaler = preprocessing.StandardScaler()
# standard_scalar = preprocessing.StandardScaler()

data_train_scaled = min_max_scaler.fit_transform(df_train_agg_imputed_features)

In [ ]:
# Visualizing the training data after imputing and aggregating

plt.figure(figsize=(16, 16))
ax = sns.boxplot(data = pd.DataFrame(data_train_scaled))
ax.set_xticklabels(
    list(df_train_features),
    rotation=90,
    horizontalalignment='right'
);

In [ ]:
# What is the correlation between the 
pd.DataFrame(data_train_scaled).corrwith(other = pd.DataFrame(df_train_agg_imputed_features), method = "spearman").transpose()

### PCA plot 

In [ ]:
pca = PCA(n_components=2)

principalComponents = pca.fit_transform(data_train_scaled)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

finalDf = pd.concat([principalDf, df_train_labels[[df_train_labels.columns[1]]]], axis = 1)

fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA for label i', fontsize = 20)
targets = [0, 1]
colors = ['r', 'g', 'b']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf[df_train_labels.columns[1]] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

### Test Data pre-processing

In [ ]:
test_pids = df_test_features['pid'].unique()
df_test_agg_features = aggregation_technique(df_test_features, df_test_features.columns.drop("pid"),
                                             test_pids, vars_classes)

In [ ]:
print(df_test_agg_features.isna().sum(axis = 0).tolist()[0:100])
print(df_train_agg_features.isna().sum(axis = 0).tolist()[0:100])

In [ ]:
# impute missing data points
# should we impute it with the same imputer that we've used for train?

imputer = KNNImputer(n_neighbors= 10)
df_test_agg_imputed_features = imputer.fit_transform(df_test_agg_features)

In [45]:
# scale test data
min_max_scaler = preprocessing.RobustScaler()
data_test_scaled = min_max_scaler.fit_transform(df_test_agg_imputed_features)

NameError: name 'df_test_agg_imputed_features' is not defined

In [ ]:
pd.DataFrame(data_train_scaled).to_csv("./Results/dat_train_scaled.csv", index = False)
pd.DataFrame(data_test_scaled).to_csv("./Results/dat_test_scaled.csv", index = False)

In [4]:
# If want to run from here:
data_train_scaled = pd.read_csv("./Results/dat_train_scaled.csv")
data_test_scaled = pd.read_csv("./Results/dat_test_scaled.csv")

## Fit a model & Predict

### predict with support vector machine classification and use probabilities

In [10]:
print(list(set(df_test_features.pid))[1:30])
test_pids = df_test_features['pid'].unique()
print(test_pids[1:30])

[3, 5, 7, 9, 11, 12, 15, 16, 17, 19, 21, 22, 25, 28, 32, 36, 40, 42, 44, 46, 52, 58, 59, 60, 65, 66, 67, 68, 72]
[10001 10003 10004 10005 10008 10011 10017 10018 10019 10025 10028  1003
 10032 10033 10039  1004 10041 10042 10043 10046 10047  1005 10051  1006
 10060 10062 10063 10064 10067]


In [41]:
# first for the labels that have an output [0,1]
columns_1 = [test_pids]


for i in range(1, 12):
    
    # feature selection
    transformer =  GenericUnivariateSelect(score_func= f_classif, mode ='k_best', param=50)
    train_features = pd.DataFrame(transformer.fit_transform(data_train_scaled, df_train_labels.iloc[:,i]))
    print("For feature ", df_train_labels.columns[i])
#     print(df_train_agg_features.columns[transformer.get_support(indices = True)])
    test_features = pd.DataFrame(transformer.transform(data_test_scaled))

#     values_1 = train_features.loc[df_train_labels[df_train_labels.columns[i]] == 1]
#     values_0 = train_features.loc[df_train_labels[df_train_labels.columns[i]] == 0]
#     values_0 = resample(values_0, replace = False, n_samples = values_1.shape[0])

#     train_features = pd.concat([values_0, values_1])
    
#     labels = np.repeat([0,1], values_0.shape[0])
    
    #clf = BaggingClassifier(SVC(kernel = 'poly', degree = 5, class_weight = 'balanced', verbose = True, C = 10))
    clf_w = SVC(kernel = 'rbf', class_weight = 'balanced', verbose = 2)
    
    parameters = {'C':(0.1, 1, 10, 20)}
    clf = model_selection.GridSearchCV(estimator= clf_w, param_grid = parameters, cv = 5,
                                        refit = True, scoring = 'roc_auc', verbose = 2,
                                       n_jobs=6, return_train_score = True)
    clf.fit(train_features, df_train_labels.iloc[:,i])
#     clf.fit(train_features, labels)
    
#     print(clf.cv_results_)
    print(clf.best_params_)
    print(clf.best_score_)
    # compute probabilites as opposed to predictions
    #dual_coefficients = clf.dual_coef_    # do we have to normalize with norm of this vector ?
    
    distance_hyperplane = clf.decision_function(test_features)
    probability = np.empty(len(distance_hyperplane))
    for j in range(0, len(probability)):
        if distance_hyperplane[j] < 0:
            probability[j] = 1 - 1/(1 + math.exp(distance_hyperplane[j]))
        else:
            probability[j] = 1/(1 + math.exp(-distance_hyperplane[j]))
    columns_1.append(probability)

    
    distance_hyperplace_train = clf.decision_function(train_features)
    probability = np.empty(len(distance_hyperplace_train))
    for j in range(0, len(probability)):
        if distance_hyperplace_train[j] < 0:
            probability[j] = 1 - 1/(1 + math.exp(distance_hyperplace_train[j]))
        else:
            probability[j] = 1/(1 + math.exp(-distance_hyperplace_train[j]))
    
    tmp = roc_auc_score(y_score= probability, y_true= df_train_labels.iloc[:,i])
    print("ROC AUC for feature", list(df_train_labels)[i] , " : ", tmp)


For feature  LABEL_BaseExcess
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  20 out of  20 | elapsed:  3.4min remaining:    0.0s
[Parallel(n_jobs=6)]: Done  20 out of  20 | elapsed:  3.4min finished


[LibSVM]{'C': 1}
0.8289746900517461
ROC AUC for feature LABEL_BaseExcess  :  0.8903110074327428
For feature  LABEL_Fibrinogen
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  20 out of  20 | elapsed:  2.1min remaining:    0.0s
[Parallel(n_jobs=6)]: Done  20 out of  20 | elapsed:  2.1min finished


[LibSVM]{'C': 0.1}
0.7467884545122396
ROC AUC for feature LABEL_Fibrinogen  :  0.799909795802379
For feature  LABEL_AST
Fitting 5 folds for each of 4 candidates, totalling 20 fits


...

[LibSVM]{'C': 0.1}
0.710027274580626
ROC AUC for feature LABEL_Bilirubin_direct  :  0.8381013812437997
For feature  LABEL_EtCO2
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  20 out of  20 | elapsed:  1.6min remaining:    0.0s
[Parallel(n_jobs=6)]: Done  20 out of  20 | elapsed:  1.6min finished


[LibSVM]{'C': 0.1}
0.8064457612490123
ROC AUC for feature LABEL_EtCO2  :  0.8640885550882911
For feature  LABEL_Sepsis
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  20 out of  20 | elapsed:  2.1min remaining:    0.0s
[Parallel(n_jobs=6)]: Done  20 out of  20 | elapsed:  2.1min finished


[LibSVM]{'C': 0.1}
0.6542154499915278
ROC AUC for feature LABEL_Sepsis  :  0.7592515886820468


In [ ]:
# labels that have a real value
columns_2 = []
# from sklearn.kernel_ridge import KernelRidge

for i in range(12, 16):
    # feature selection
    transformer =  GenericUnivariateSelect(score_func= mutual_info_regression, mode ='k_best', param =80)
    train_features = transformer.fit_transform(data_train_scaled, df_train_labels.iloc[:,i])
#     print(df_train_agg_features.columns[transformer.get_support(indices = True)])
    test_features = transformer.transform(data_test_scaled)
    
    clf_w = SVR(kernel = 'rbf', cache_size = 6000)
# #     clf_w = NuSVR(nu=0.5, kernel = 'linear')
    parameters = {'C':(0.1, 1,10, 20)}
    clf = model_selection.GridSearchCV(estimator= clf_w, param_grid = parameters, cv = 10,
                                       refit = True, scoring = 'r2', verbose = 2, n_jobs=6)
#     clf = KernelRidge(kernel = 'poly', degree = 5)
#     parameters = {'alpha':(0.1,1,10,30)}
#     clf = model_selection.GridSearchCV(estimator= clf, param_grid = parameters, cv = 3,
#                                       refit = True, scoring = 'r2', verbose = 2, n_jobs=6)
    clf.fit(train_features, df_train_labels.iloc[:,i])
    
#     print(clf.cv_results_)
    print(clf.best_params_)
    print(clf.best_score_)

    pred_train = clf.predict(train_features)
    tmp = r2_score(y_pred= pred_train, y_true=df_train_labels.iloc[:,i])
    print("R2 for feature", list(df_train_labels)[i] , " : ", tmp)
    
    pred = clf.predict(test_features)
    columns_2.append(pred)
    

In [ ]:
columns_final = columns_1 + columns_2

### predict with Support vector regression and then compute sigmoid function

In [ ]:
# first for the labels that have an output [0,1]

# columns_1 = [test_pids]

# for i in range(1,12):
    
#     clf = SVR(kernel = 'poly', degree = 3, max_iter = 10000)
#     clf.fit(data_train_scaled, df_train_labels.iloc[:,i])
#     pred = clf.predict(data_test_scaled)
#     prob = np.empty(len(pred))
#     for j in range(0, len(pred)):
#         prob[j] = 1 / (1 + math.exp(-pred[j]))
#     columns_1.append(prob)
    
#     pred_train = clf.predict(data_train_scaled)
#     prob_train = np.empty(len(pred_train))
#     for j in range(0, len(pred_train)):
#         prob_train[j] = 1 / (1 + math.exp(-pred_train[j]))    
#     tmp = roc_auc_score(y_score= prob_train, y_true= df_train_labels.iloc[:,i])
#     print("ROC AUC for feature", list(df_train_labels)[i] , " : ", tmp)


In [42]:
#labels that have a real value

columns_2 = []

for i in range(12, 16):
    
    # feature selection
    transformer =  GenericUnivariateSelect(score_func= mutual_info_regression, mode ='k_best', param=80)
    train_features = transformer.fit_transform(data_train_scaled, df_train_labels.iloc[:,i])
#     print(list(data_train_scaled)[transformer.get_support()])
    test_features = transformer.transform(data_test_scaled)
    
    if(i == 14):
        clf_w = SVR(kernel = 'rbf', cache_size = 6000)
        parameters = {'C':(0.1, 1,10, 20)}
    else:
        clf_w = LinearSVR()
        parameters = {'C':(0.01, 0.1,1,10,30,50, 75,100)}
        
    clf = model_selection.GridSearchCV(estimator= clf_w, param_grid = parameters, cv = 10,
                                       refit = True, scoring = 'r2', verbose = 1, n_jobs=6)
    clf.fit(train_features, df_train_labels.iloc[:,i])
    
    print(clf.best_params_)
    print(clf.best_score_)
    
    pred = clf.predict(test_features)
    columns_2.append(pred)
    
    pred_train = clf.predict(train_features)
    tmp = r2_score(y_pred= pred_train, y_true=df_train_labels.iloc[:,i])
    print("R2 for feature", list(df_train_labels)[i] , " : ", tmp)

Fitting 10 folds for each of 8 candidates, totalling 80 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
/usr/local/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=6)]: Done  56 tasks      | elapsed:   48.6s
[Parallel(n_jobs=6)]: Done  80 out of  80 | elapsed:  1.3min finished


{'C': 0.1}
0.4272675402863789
R2 for feature LABEL_RRate  :  0.43301788996082624
Fitting 10 folds for each of 8 candidates, totalling 80 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  56 tasks      | elapsed:   38.9s
[Parallel(n_jobs=6)]: Done  80 out of  80 | elapsed:  1.1min finished


{'C': 0.1}
0.5976017817154319
R2 for feature LABEL_ABPm  :  0.6193099863826221
Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  40 out of  40 | elapsed:  7.6min finished


{'C': 1}
0.2990090520477443
R2 for feature LABEL_SpO2  :  0.41787686694537984
Fitting 10 folds for each of 8 candidates, totalling 80 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  56 tasks      | elapsed:   42.1s
[Parallel(n_jobs=6)]: Done  80 out of  80 | elapsed:  1.2min finished


{'C': 0.1}
0.6153758143241128
R2 for feature LABEL_Heartrate  :  0.6554719255181802


In [ ]:
transformer =  GenericUnivariateSelect(score_func= mutual_info_regression, mode ='k_best', param=20)
train_features = transformer.fit_transform(data_train_scaled, df_train_labels.iloc[:,11])
test_features = transformer.transform(data_test_scaled)

In [ ]:
df_train_agg_features.columns[transformer.get_support(indices = True)]

In [43]:
columns_final = columns_1 + columns_2

### Random forest

In [ ]:
# Random forest Classifier
columns_1 = [test_pids]
for i in range(1, 12):
    clf = RandomForestClassifier(min_samples_leaf=2, class_weight='balanced', oob_score=False, bootstrap=False)
    clf.fit(data_train_scaled, df_train_labels.iloc[:,i])
    print(clf.oob_score)
    # compute probabilites as opposed to predictions
    probability = clf.apply(data_test_scaled)
    probs = [i[1] for i in probability] 
    columns_1.append(probs)
    
    
    probability = clf.predict_proba(data_train_scaled)

    probs = [i[1] for i in probability]            
    tmp = roc_auc_score(y_score= probs, y_true= df_train_labels.iloc[:,i])
    print("ROC AUC for feature", list(df_train_labels)[i] , " : ", tmp)

# Kernelized Logistic Regression

In [34]:
# first for the labels that have an output [0,1]
columns_1 = [test_pids]

from sklearn.linear_model import LogisticRegression
from sklearn.kernel_approximation import Nystroem
from sklearn.model_selection import StratifiedKFold, KFold

for i in range(1, 12):
    i = 11
    #     feature selection
#     transformer =  GenericUnivariateSelect(score_func= mutual_info_classif, mode ='k_best', param = 80)
#     train_features = transformer.fit_transform(data_train_scaled, df_train_labels.iloc[:,i])
# #     print(df_train_agg_features.columns[transformer.get_support(indices = True)])
#     test_features = transformer.transform(data_test_scaled)

    feature_map_nystroem = Nystroem(kernel = 'rbf',
                                 random_state=1,
                                 n_components=350)
    
    train_transformed = feature_map_nystroem.fit_transform(data_train_scaled)
    test_transformed = feature_map_nystroem.transform(data_test_scaled)
    
    clf_w = LogisticRegression(penalty = 'l2', class_weight = 'balanced', max_iter=10000)
    
    # checked before
    #parameters = {'alpha':(0.0001, 0.001, 0.01, 0.1, 1, 5, 10, 20, 30)}
    parameters = {'C':(0.01, 0.1, 1, 5, 10, 20, 30)}
    
    skf = StratifiedKFold(n_splits=10, shuffle = True, random_state = 1)
    
    clf = model_selection.GridSearchCV(estimator= clf_w, param_grid = parameters, cv = skf,
                                        refit = True, scoring = 'roc_auc', verbose = 1,
                                       n_jobs=6, return_train_score = True)
    clf.fit(train_transformed, df_train_labels.iloc[:,i])
    
    print(clf.cv_results_['mean_train_score'])
    print(clf.cv_results_['mean_test_score'])
    print(clf.best_params_)
    print(clf.best_score_)
 
    probability_tmp = clf.predict_proba(test_transformed)
    probability = [item[1] for item in probability_tmp]
    columns_1.append(probability)

    probability_tmp = clf.predict_proba(train_transformed)
    probability_train = [item[1] for item in probability_tmp]
    tmp = roc_auc_score(y_score= probability_train, y_true= df_train_labels.iloc[:,i])
    print("ROC AUC for feature", list(df_train_labels)[i] , " : ", tmp)

Fitting 10 folds for each of 7 candidates, totalling 70 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:  3.0min
[Parallel(n_jobs=6)]: Done  70 out of  70 | elapsed: 11.0min finished


[0.6626445  0.69937407 0.73216233 0.7475475  0.75114156 0.75327248
 0.75441509]
[0.65299292 0.67255175 0.67248679 0.66760654 0.65922669 0.65823246
 0.65261516]
{'base_estimator__C': 0.1}
0.6725517455777675
ROC AUC for feature LABEL_Sepsis  :  0.6980688520591686
Fitting 10 folds for each of 7 candidates, totalling 70 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:  3.1min


KeyboardInterrupt: 

# Compute the kernel and use SGD Classifier and Regressor

In [ ]:
# first for the labels that have an output [0,1]
columns_1 = [test_pids]

# from sklearn.kernel_ridge import KernelRidge
from sklearn.kernel_approximation import Nystroem
from sklearn import linear_model


for i in range(1, 12):
   
    # feature selection
    transformer =  GenericUnivariateSelect(score_func= mutual_info_classif, mode ='k_best', param=70)
    train_features = transformer.fit_transform(data_train_scaled, df_train_labels.iloc[:,i])
    print("For feature ", df_train_labels.columns[i])
#     print(df_train_agg_features.columns[transformer.get_support(indices = True)])
    test_features = transformer.transform(data_test_scaled)

    
    feature_map_nystroem = Nystroem(kernel = 'poly', degree = 3,
                                 random_state=1,
                                 n_components=300)
    train_transformed = feature_map_nystroem.fit_transform(train_features)
    test_transformed = feature_map_nystroem.transform(test_features)
    
    clf_w = linear_model.SGDClassifier(max_iter=100000, tol=1e-4, penalty = "l2", 
                                       loss = "epsilon_insensitive", class_weight='balanced')
    # checked before
    #parameters = {'alpha':(0.0001, 0.001, 0.01, 0.1, 1, 5, 10, 20, 30)}
    parameters = {'alpha':(0.1, 1, 5, 10)}
    
    clf = model_selection.GridSearchCV(estimator= clf_w, param_grid = parameters, cv = 10,
                                        refit = True, scoring = 'roc_auc', verbose = 1,
                                       n_jobs=6, return_train_score = True)
    clf.fit(train_transformed, df_train_labels.iloc[:,i])
    
#     print(clf.cv_results_)
    print(clf.best_params_)
    print(clf.best_score_)
    # compute probabilites as opposed to predictions
    #dual_coefficients = clf.dual_coef_    # do we have to normalize with norm of this vector ?
    
    distance_hyperplane = clf.decision_function(test_transformed)
    probability = np.empty(len(distance_hyperplane))
    for j in range(0, len(probability)):
        if distance_hyperplane[j] < 0:
            probability[j] = 1 - 1/(1 + math.exp(distance_hyperplane[j]))
        else:
            probability[j] = 1/(1 + math.exp(-distance_hyperplane[j]))
    columns_1.append(probability)

    
    distance_hyperplace_train = clf.decision_function(train_transformed)
    probability = np.empty(len(distance_hyperplace_train))
    for j in range(0, len(probability)):
        if distance_hyperplace_train[j] < 0:
            probability[j] = 1 - 1/(1 + math.exp(distance_hyperplace_train[j]))
        else:
            probability[j] = 1/(1 + math.exp(-distance_hyperplace_train[j]))
    
    tmp = roc_auc_score(y_score= probability, y_true= df_train_labels.iloc[:,i])
    print("ROC AUC for feature", list(df_train_labels)[i] , " : ", tmp)
    

In [ ]:
# labels that have a real value
columns_2 = []

for i in range(12, 16):
#     feature selection
    transformer =  GenericUnivariateSelect(score_func= mutual_info_regression, mode ='k_best', param = 40)
    train_features = transformer.fit_transform(data_train_scaled, df_train_labels.iloc[:,i])
#     print(df_train_agg_features.columns[transformer.get_support(indices = True)])
    test_features = transformer.transform(data_test_scaled)
    
    feature_map_nystroem = Nystroem(kernel = 'rbf',
                                 random_state=1,
                                 n_components=200)
    train_features = feature_map_nystroem.fit_transform(train_features)
    test_features = feature_map_nystroem.transform(test_features)
    
    clf_w = linear_model.SGDRegressor(max_iter=100000, tol=1e-4,
                                     loss = 'epsilon_insensitive', penalty = 'l2')
    parameters = {'alpha':(0.0001, 0.001, 0.01, 0.1, 1, 5, 10)}
    clf = model_selection.GridSearchCV(estimator= clf_w, param_grid = parameters, cv = 10,
                                       refit = True, scoring = 'r2', verbose = 1, n_jobs=6, return_train_score=True)
#     clf = KernelRidge(kernel = 'poly', degree = 5)
#     parameters = {'alpha':(0.1,1,10,30)}
#     clf = model_selection.GridSearchCV(estimator= clf, param_grid = parameters, cv = 3,
#                                       refit = True, scoring = 'r2', verbose = 2, n_jobs=6)
    clf.fit(train_features, df_train_labels.iloc[:,i])
    
#     print(clf.cv_results_)
    print(clf.cv_results_['mean_train_score'])
    print(clf.cv_results_['mean_test_score'])
    print(clf.best_params_)
    print(clf.best_score_)
    
    pred = clf.predict(test_features)
    columns_2.append(pred) 

    pred_train = clf.predict(train_features)
    tmp = r2_score(y_pred= pred_train, y_true=df_train_labels.iloc[:,i])
    print("R2 for feature", list(df_train_labels)[i] , " : ", tmp)
   

In [ ]:
columns_final = columns_1 + columns_2

# XGBoosting

The benefit of XGBoost is that it accepts missing values.

In [ ]:
import xgboost as xgb
# simple example
# load file from text file, also binary buffer generated by xgboost

# scale test data
min_max_scaler = preprocessing.StandardScaler()
df_train_scaled_xgboost = min_max_scaler.fit_transform(df_train_agg_features)
min_max_scaler = preprocessing.StandardScaler()
df_test_scaled_xgboost = min_max_scaler.fit_transform(df_test_agg_features)

dtrain = xgb.DMatrix(df_train_scaled_xgboost, label=df_train_labels.iloc[:,1])
dtest = xgb.DMatrix(df_test_scaled_xgboost)

In [ ]:
param = {'eta': 1, 'objective': 'binary:logistic', 'verbosity':1}
param['nthread'] = 4
param['eval_metric'] = 'auc'
evallist = [(dtrain, 'train')]

In [ ]:
num_round = 10
xgb.cv(param, dtrain, num_round, nfold=10,
       metrics={'auc'}, seed=0,
       callbacks=[xgb.callback.print_evaluation(show_stdv=True)])
# bst = xgb.train(param, dtrain, num_round)
# pred = bst.predict(dtest)

In [24]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.kernel_approximation import Nystroem

columns_1 = [test_pids]


for i in range(1,12):

    clf = xgb.XGBClassifier('binary:logistic')
    
    clf_w = model_selection.GridSearchCV(estimator= clf, param_grid = {'max_depth': [2,3,4,5]}, cv = 10,
                                        refit = True, scoring = 'roc_auc', verbose = 1,
                                       n_jobs=6, return_train_score = True)
    

    clf_w.fit(data_train_scaled, df_train_labels.iloc[:,i])
    print(clf_w.best_params_)
    print(clf_w.best_score_) 
#     dtrain = xgb.DMatrix(df_train_agg_features, label=df_train_labels.iloc[:,i])
#     num_round = 10
#     xgb.cv(param, dtrain, num_round, nfold=10,
#        metrics={'auc'}, seed=0,
#        callbacks=[xgb.callback.print_evaluation(show_stdv=True)])
    
    probability = clf_w.predict(data_test_scaled)
    columns_1.append(probability)

    probability_train = clf_w.predict(data_train_scaled)
#     probability_train = [item[1] for item in probability_tmp]
    tmp = roc_auc_score(y_score= probability_train, y_true= df_train_labels.iloc[:,i])
    print("ROC AUC for feature", list(df_train_labels)[i] , " : ", tmp)

Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  40 out of  40 | elapsed:  2.0min finished


{'max_depth': 4}
0.9021221528886334
ROC AUC for feature LABEL_BaseExcess  :  0.8789192323561444
Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  40 out of  40 | elapsed:  1.9min finished


{'max_depth': 2}
0.7655657229886372
ROC AUC for feature LABEL_Fibrinogen  :  0.6207097186700767
Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  40 out of  40 | elapsed:  2.0min finished


{'max_depth': 2}
0.7111729556295098
ROC AUC for feature LABEL_AST  :  0.5960129151503049
Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


KeyboardInterrupt: 

In [19]:
columns_2_1 = []
param = {'max_depth': 2, 'eta': 1, 'objective': 'reg:squarederror', 'verbosity':1}
param['nthread'] = 4
param['eval_metric'] = 'rmse'
# evallist = [(dtrain, 'train')]

train_features = data_train_scaled
test_features = data_test_scaled
for i in range(12,16):
#     transformer =  GenericUnivariateSelect(score_func= mutual_info_regression, mode ='k_best', param=80)
#     train_features = transformer.fit_transform(data_train_scaled, df_train_labels.iloc[:,i])
# #     print(list(data_train_scaled)[transformer.get_support()])
#     test_features = transformer.transform(data_test_scaled)
    
    clf_w = xgb.XGBRegressor(feval = r2_score, verbosity = 0)
            
    clf = model_selection.GridSearchCV(estimator= clf_w, param_grid = {'max_depth':(3,4,5)}, cv = 10,
                                       refit = True, verbose = 1, scoring='r2',
                                       n_jobs=6, return_train_score = True)
    
    clf.fit(train_features, df_train_labels.iloc[:,i])
    print(clf.best_params_)
    print(clf.best_score_)    
    
#     dtrain = xgb.DMatrix(df_train_scaled_xgboost, label=df_train_labels.iloc[:,i])
#     num_round = 10
    
    probability = clf.predict(test_features)
    columns_2_1.append(probability)

    pred_train = clf.predict(train_features)
#     probability_train = [item[1] for item in probability_tmp]
    tmp = r2_score(y_pred= pred_train, y_true=df_train_labels.iloc[:,i])
    print("R2 for feature", list(df_train_labels)[i] , " : ", tmp)
   

Fitting 10 folds for each of 3 candidates, totalling 30 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  30 out of  30 | elapsed:   51.6s finished


{'max_depth': 3}
0.4200176924297615
R2 for feature LABEL_RRate  :  0.5392544663130121
Fitting 10 folds for each of 3 candidates, totalling 30 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  30 out of  30 | elapsed:   54.7s finished


{'max_depth': 3}
0.6144187790982285
R2 for feature LABEL_ABPm  :  0.6863046639629317
Fitting 10 folds for each of 3 candidates, totalling 30 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  30 out of  30 | elapsed:   53.4s finished


{'max_depth': 3}
0.3421986139455796
R2 for feature LABEL_SpO2  :  0.607600007016429
Fitting 10 folds for each of 3 candidates, totalling 30 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  30 out of  30 | elapsed:   54.0s finished


{'max_depth': 3}
0.6576084499489625
R2 for feature LABEL_Heartrate  :  0.7175087922391096


In [20]:
columns_final = columns_1 + columns_2_1

## Save predictions

In [44]:
print(np.shape(columns_final))
result = pd.DataFrame(columns_final).transpose()
result.columns = list(df_train_labels)
result.to_csv('./Results/prediction.csv.zip', index=False, float_format='%.3f', compression='zip')

(16, 12664)


In [ ]:
result.to_csv('./Results/prediction.csv', index=False, float_format='%.3f')